### SimpleMOC

Compiler: clang  
flags: `-g -pg -Ofast -ffast-math -ftree-vectorize`  
libs: `-lm`  
On Intel(R) Xeon(R) CPU E5-2699 v4 @ 2.20GHz (Broadwell)

1.07e10 usec cputime

Work done in `solver.c`, within function `attenuate_fluxes`:  
Function Structure for Context:

1. Declare/Initialize Variables. Including 
```c
// compute fine axial region ID
        int fine_id = (int) ( track->z_height / dz ) % I.fai;
```
2. if/ else if / else Control
```c
if( fine_id == 0 ) {adjustment and loop};
else if ( fine_id == I.fai - 1 ) {adjustment and loop};
else // loop at line 121 (first show below)
```
3. 11 small for loops with 1 variable declaration in the middle (Three of the busiest shown below)

#### else Body

loop at solver.c: line 121 : 9.3%
```c
		for( int g = 0; g < I.n_egroups; g++)
		{
			// load neighboring sources
			float y1 = QSR->fine_source[fine_id-1][g];
			float y2 = QSR->fine_source[fine_id][g];
			float y3 = QSR->fine_source[fine_id+1][g];

			// do quadratic "fitting"
			float c0 = y2;
			float c1 = (y1 - y3) / (2.f*dz);
			float c2 = (y1 - 2.f*y2 + y3) / (2.f*dz*dz);

			// calculate q0, q1, q2
			q0[g] = c0 + c1*zin + c2*zin*zin;
			q1[g] = c1 + 2.f*c2*zin;
			q2[g] = c2;
		}
```

#### Heavy Loops

loop at solver.c : line 162 : 8.2%
```c
	for( int g = 0; g < I.n_egroups; g++)
		expVal[g] = interpolateTable( params.expTable, tau[g] );  
```

loop at solver.c : line 195 : 8.7%
```c
	for( int g = 0; g < I.n_egroups; g++)
	{
		// add contribution to new source flux
		flux_integral[g] = (q0[g] * tau[g] + (sigT[g] * psi[g] - q0[g])
			* expVal[g]) / sigT2[g] + q1[g] * mu * reuse[g] + q2[g] * mu2 
			* (tau[g] * (tau[g] * (tau[g] - 3.f) + 6.f) - 6.f * expVal[g]) 
			/ (3.f * sigT2[g] * sigT2[g]);
	}
```

loop at solver.c : line 223 : 7.6%
```c
	for( int g = 0; g < I.n_egroups; g++)
	{
		FSR_flux[g] += tally[g];
	}
```

266 Total lines in function, lots of preproccessor options along with good amount of comments and whitespace.  Might be able to fit all the code.